<a href="https://colab.research.google.com/github/sfarrukhm/smollm-fine-tuning/blob/main/smollm_sft_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU trl transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from huggingface_hub import login
login()

In [3]:
from datasets import load_dataset
# from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

datasets=load_dataset("HuggingFaceTB/smol-smoltalk")

README.md:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/230M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/231M [00:00<?, ?B/s]

train-00003-of-00004.parquet:   0%|          | 0.00/232M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/48.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/460341 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/24229 [00:00<?, ? examples/s]

In [4]:
len(datasets['train'][17000]['messages'])

6

In [5]:
datasets['train'][0]['messages'][1]

{'content': 'Here\'s a revised version of your text with some suggested edits to improve grammar, clarity, and overall flow:\n\n"My friends and I have been waiting a long time to return to the movies and see a film we\'ve been eagerly anticipating. Last night, I finally went to buy tickets, but when I arrived at the theater, they were sold out. I was pretty upset, and my friends were too. Today, we discussed the possibility of going to a game instead, but I don\'t think we\'ll end up going."\n\nI made the following changes:\n\n- Changed "Me and my friends" to "My friends and I" to correct subject-verb order.\n- Changed "catch a movie we been waiting on" to "see a film we\'ve been eagerly anticipating" to use more precise and polished language.\n- Changed "Last nite" to "Last night" to correct spelling.\n- Changed "finely" to "finally" to correct spelling and usage.\n- Changed "pist" to "upset" to use a more common and suitable word for the context.\n- Changed "deside" to "discussed" to

In [6]:
datasets['train'][0]['messages'][2]

{'content': "What is the meaning of word 'finely' in English language?",
 'role': 'user'}

In [7]:
datasets['train'][0]['messages'][3]

{'content': 'In the English language, "finely" is an adverb that has several meanings. Here are a few possible interpretations:\n\n1. **In a delicate or precise manner**: "Finely" can describe an action or process that is done with great care, subtlety, or attention to detail. For example: "The chef chopped the herbs finely to release their flavors."\n\n2. **To a fine or smooth texture**: "Finely" can also describe the texture or consistency of something, often implying that it is smooth, powdery, or free from coarseness. For example: "The sand on the beach was finely ground, making it perfect for building sandcastles."\n\n3. **With refinement or elegance**: In some cases, "finely" can convey a sense of refinement, sophistication, or elegance. For example: "The finely crafted furniture in the mansion reflected the owner\'s exquisite taste."\n\nHowever, in the original text you provided, "finely" was likely used incorrectly, and I suggested replacing it with "finally", which means "at l

In [8]:
datasets['train'][0]['messages'][4]

{'content': 'In a town with 100 people 90 of them hate a new policy in the town so how many people are for it.',
 'role': 'user'}

In [9]:
datasets['train'][0]['messages'][2]

{'content': "What is the meaning of word 'finely' in English language?",
 'role': 'user'}

In [10]:
# Load the base model
from transformers import AutoModelForCausalLM, AutoTokenizer
base_checkpoint="HuggingFaceTB/SmolLM2-135M"
tokenizer= AutoTokenizer.from_pretrained(base_checkpoint)
base_model=AutoModelForCausalLM.from_pretrained(base_checkpoint)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [11]:
prompt="In a town with 100 people 90 of them hate a new policy in the town so how many people are for it."
input_tokens=tokenizer(prompt,return_tensors='pt')
outputs=base_model(**input_tokens)
logits=outputs.logits
out_tokens=torch.argmax(logits, dim=-1)
tokenizer.decode(out_tokens[0])

' the  of a100,,10% them are the certain movie. the city hall they can people are actually the?\n'

In [12]:
base_model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "HuggingFaceTB/SmolLM2-135M",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "eos_token_id": 0,
  "head_dim": 64,
  "hidden_act": "silu",
  "hidden_size": 576,
  "initializer_range": 0.041666666666666664,
  "intermediate_size": 1536,
  "is_llama_config": true,
  "max_position_embeddings": 8192,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 9,
  "num_hidden_layers": 30,
  "num_key_value_heads": 3,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_interleaved": false,
  "rope_scaling": null,
  "rope_theta": 100000,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.48.1",
  "use_cache": true,
  "vocab_size": 49152
}

In [15]:
# testing Base model using chat template
from trl import SFTConfig, SFTTrainer, setup_chat_format
model, tokenizer=setup_chat_format(base_model, tokenizer )

#Augment the prompt with chat template
messages=[{"role":"user","content":prompt}]
augmented_prompt=tokenizer.apply_chat_template(messages,tokenize=False)
print(augmented_prompt)
# tokenize the formated prompt
inputs=tokenizer(augmented_prompt,return_tensors="pt")
tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])



<|im_start|>user
In a town with 100 people 90 of them hate a new policy in the town so how many people are for it.<|im_end|>



['<|im_start|>',
 'user',
 'Ċ',
 'In',
 'Ġa',
 'Ġtown',
 'Ġwith',
 'Ġ',
 '1',
 '0',
 '0',
 'Ġpeople',
 'Ġ',
 '9',
 '0',
 'Ġof',
 'Ġthem',
 'Ġhate',
 'Ġa',
 'Ġnew',
 'Ġpolicy',
 'Ġin',
 'Ġthe',
 'Ġtown',
 'Ġso',
 'Ġhow',
 'Ġmany',
 'Ġpeople',
 'Ġare',
 'Ġfor',
 'Ġit',
 '.',
 '<|im_end|>',
 'Ċ']

In [16]:
# Generate output
outputs=model.generate(**inputs)
print("Model output before training: ")
print(tokenizer.decode(outputs[0],skip_special_tokens=True))

Model output before training: 
user
In a town with 100 people 90 of them hate a new policy in the town so how many people are for it.
In a town with 100 people 90 of them hate a new policy in the


In [17]:
tokenizer.decode(outputs[0])

'<|im_start|>user\nIn a town with 100 people 90 of them hate a new policy in the town so how many people are for it.<|im_end|>\nIn a town with 100 people 90 of them hate a new policy in the'

### Step training configuration and start training

In [19]:
# first of all downsample the dataset
subset = datasets["train"].select(range(50000))  # Extract first 50K rows
downsampled_datasets=subset.train_test_split(test_size=0.2, seed=42)  # 80% train, 20% test
downsampled_datasets


DatasetDict({
    train: Dataset({
        features: ['messages', 'source'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['messages', 'source'],
        num_rows: 10000
    })
})

In [20]:
from trl import SFTConfig, SFTTrainer

output_dir="/content/drive/MyDrive/smollm-fine-tuning/trained_models"
model_hub_name="smollm-smtalk-v1"
sft_config=SFTConfig(
    output_dir=output_dir,
    max_steps=1000,
    per_device_train_batch_size=4,
    learning_rate=2e-5,
    logging_steps=10,
    save_steps=1000,
    evaluation_strategy="epoch",
    hub_model_id=model_hub_name

)

trainer=SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=downsampled_datasets['train'],
    eval_dataset=downsampled_datasets["test"],
    processing_class=tokenizer
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [21]:
trainer.train()
trainer.push_to_hub("V1")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


...

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1737440877.4adc5e788463.806.0:   0%|          | 0.00/27.4k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sfarrukh/smollm-smtalk-v1/commit/8b8ce4b9b4d257816af3dd6df49e16dd155df869', commit_message='V1', commit_description='', oid='8b8ce4b9b4d257816af3dd6df49e16dd155df869', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sfarrukh/smollm-smtalk-v1', endpoint='https://huggingface.co', repo_type='model', repo_id='sfarrukh/smollm-smtalk-v1'), pr_revision=None, pr_num=None)

In [27]:
from transformers import pipeline

question = "If you had a time machine, but could only go to the past or the future once and never return, which would you choose and why?"
generator = pipeline("text-generation", model="sfarrukh/smollm-smtalk-v1", device="cuda")
output = generator([{"role": "user", "content": question}], max_new_tokens=128, return_full_text=False)[0]
print(output["generated_text"])


Device set to use cuda


I would choose the past, because it's the only way to understand the world and the people who lived in it. The past is the only place where I can truly grasp the complexities of history, the intricacies of human behavior, and the human condition. The past is where I can see the world from the perspective of the people who lived it, and where I can experience the emotions and experiences that shaped their lives.

The future, on the other hand, is where I can see the world from the perspective of the people who will live it, and where I can experience the emotions and experiences that will shape their lives. The
